In [1]:
import sys
sys.path.append("/home/files/uos_plantclassification")
import pandas as pd
import json
from os.path import join
from os import listdir
import albumentations as A
from albumentations.pytorch import ToTensorV2
from config.path import PATH
from data import get_mini_plantnet, MiniPlantNet, PlantNet300K
transforms = {
    'train': A.Compose([
        A.Resize(height=380, width=380),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=0.0, std=1.0),
        ToTensorV2()]),
    'val': A.Compose([
        A.Resize(height=380, width=380),
        A.Normalize(mean=0.0, std=1.0),
        ToTensorV2()]),
    'test': A.Compose([
        A.Resize(height=380, width=380),
        A.Normalize(mean=0.0, std=1.0),
        ToTensorV2()])
}

dataset = PlantNet300K(root=PATH["PLANTNET-300K"], split="train", shuffle=False, transform=transforms["test"])
#dataset = MiniPlantNet(root=PATH["PLANTNET-300K"], split="train", shuffle=False, transform=transforms["test"])

In [2]:
genera_to_species = {}

for name, label in zip(dataset.name_to_label.keys(), dataset.name_to_label.values()):
    genera = name.split(" ")[0]
    if genera in genera_to_species.keys():
        genera_to_species[genera].append(name)
    else:
        genera_to_species[genera] = [name]

with open("genera_to_species.json", 'w', encoding='utf-8') as f: 
    json.dump(genera_to_species, f, ensure_ascii=False, indent=2)

In [3]:
species_to_fine = {}
for name in dataset.name_to_label:
    species_to_fine[name] = dataset.label_to_class[dataset.name_to_label[name]]
with open("species_to_fine.json", 'w', encoding='utf-8') as f: 
    json.dump(species_to_fine, f, ensure_ascii=False, indent=2)


genera_to_coarse = {}
for id, genera in enumerate(genera_to_species.keys()):
    genera_to_coarse[genera] = id
with open("genera_to_coarse.json", 'w', encoding='utf-8') as f: 
    json.dump(genera_to_coarse, f, ensure_ascii=False, indent=2)

In [4]:
fine_to_coarse = {}
for genera, species in zip(genera_to_species.keys(), genera_to_species.values()):
    coarse = genera_to_coarse[genera]
    for s in species:
        fine = species_to_fine[s]
        fine_to_coarse[fine] = coarse  
with open("fine_to_coarse.json", 'w', encoding='utf-8') as f: 
    json.dump(fine_to_coarse, f, ensure_ascii=False, indent=2)

In [5]:
#mini plantnet의 학습데이터 수
label_to_train_samples = {}
name_to_train_samples = {}
for name, label in zip(dataset.name_to_label.keys(), dataset.name_to_label.values()):
    num_samples = len(listdir(join(PATH["PLANTNET-300K"], "images", "train", label)))
    label_to_train_samples[label] = num_samples
    name_to_train_samples[name] = num_samples
with open("name_to_train_samples.json", 'w', encoding='utf-8') as file:
    json.dump(name_to_train_samples, file, ensure_ascii=False, indent=2)